# Transfer Learning.

With 'torchvision.models' we can use pre-trained models.

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, models

from collections import OrderedDict

## Loading Cat vs Dog data.

Most models requires input size 244x244. Also match the normalizations.

In [ ]:
data_dir = 'Data/DogsVsCats'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

## Loading pre-trained model. 

In [ ]:
model = models.densenet121(pretrained=True)
print(model)

## Setting up model.

In [ ]:
# Turn off gradients of our model
for params in model.parameters():
    params.requires_grad = False

In [ ]:
# Define a new classifier
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 500)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(500, 2)),
    ('output', nn.LogSoftmax(dim=1))        
]))

model.classifier = classifier

In [ ]:
# Loss function
criterion = nn.NLLLoss()

# Optimizer
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

Training that deep networks require computation power.

So, we use GPU to train these models.

In [ ]:
# Setting device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tranfer model to device
model.to(device)

## Training.

In [ ]:
epochs = 1
train_losses, validation_losses = [], []

for epoch in range(epochs):
    # Training
    for images, labels in trainloader:

        images, labels = images.to(device), labels.to(device)

        logps = model(images)

        loss = criterion(logps, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
    
    # Validation
    else:

        accuracy = 0
        with torch.no_grad():
            for images, labels in testloader:
                logps = model(images)
                test_loss = criterion(logps, labels)

                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        train_losses.append(loss)
        validation_losses.append(validation_loss)

        print(f"Epoch: {epoch+1}/{epochs}:\n",
            f"Training loss: {loss}\n",
            f"Validation loss: {validation_loss}\n",
            f"Accuracy: {accuracy/len(validationloader)}")